In [1]:
import xarray as xr
import geopandas as gpd
import networkx as nx
import pandas as pd
import numpy as np

In [2]:
upstream_seg_list= '../../geospacial/misc/milk_upstream_seg.txt'

In [3]:
geodata= pd.read_csv('../../model/model_versions/v_7/v7_1/hds_model/GeoData.txt', index_col=0, sep='\t')

In [4]:
geodata['depression_area']= geodata['area'] * geodata['SLC_62']

In [5]:
# Convert columns to string data type
geodata['maindown'] = geodata['maindown'].astype(str)
geodata.index= geodata.index.astype(str)

In [6]:
# HYPE river segments to be analyzed
ids = [
    '58223', '58213', '58208', '58408', '58643', '58308', 
    '58346', '58425', '58356', '58363', '58418', '58290', 
    '58328', '58292', '58398'
]

In [7]:
# HYPE river segments to be analyzed
stm = [
    '58223', '58213', '58208'
]

In [8]:
# HYPE river segments to be analyzed
milk = [
    '58408', '58643', '58308', 
    '58346', '58425', '58356', '58363', '58418', '58290', 
    '58328', '58292', '58398'
]

In [9]:
# Assuming `geodata` is already defined
# Create a directed graph
riv_graph = nx.DiGraph()

# Add edges from DataFrame
for idx, row in geodata.iterrows():
    if row['maindown'] != '0':  # Skip if maindown is '0'
        riv_graph.add_edge(idx, row['maindown'])

# Initialize lists to store results
ids_results = []
total_shape_areas_km2 = []
total_depression_areas_km2 = []
depression_percentages = []
upstream_seg= []

# Analyze each segment in ids
for riv_seg in milk:
    # Find upstream segments for the given segment
    upstream_segments = list(nx.ancestors(riv_graph, riv_seg))

    print(f'{riv_seg}: {upstream_segments}')

    # Add the target segment 'riv_seg' to the upstream segments
    upstream_segments.append(riv_seg)

    upstream_seg.append(upstream_segments)

    # Filter geodata for upstream segments
    result = geodata[geodata.index.isin(upstream_segments)][['area', 'depression_area']]

    # Calculate the sum of areas
    total_shape_area_m2 = result['area'].sum()
    total_depression_area_m2 = result['depression_area'].sum()

    # Convert from square meters to square kilometers
    total_shape_area_km2 = total_shape_area_m2 / 1e6
    total_depression_area_km2 = total_depression_area_m2 / 1e6

    # Calculate depression percentage
    depression_percentage = (total_depression_area_km2 / total_shape_area_km2) * 100

    # Store results
    ids_results.append(riv_seg)
    total_shape_areas_km2.append(total_shape_area_km2)
    total_depression_areas_km2.append(total_depression_area_km2)
    depression_percentages.append(depression_percentage)

# Combine results into a DataFrame for easier viewing
results_df = pd.DataFrame({
    'Segment ID': ids_results,
    'Total Shape Area (km²)': total_shape_areas_km2,
    'Total Depression Area (km²)': total_depression_areas_km2,
    'Depression Percentage (%)': depression_percentages
})

results_df

58408: ['58661', '58434', '58609', '58658', '58508', '58607', '58498']
58643: []
58308: ['58639', '58377', '58485', '58623', '58633', '58620', '58434', '58319', '58495', '58331', '58543', '58507', '58421', '58658', '58520', '58323', '58607', '58649', '58557', '58385', '58498', '58661', '58567', '58626', '58325', '58521', '58544', '58508', '58630', '58309', '58578', '58609', '58322', '58645', '58335', '58379', '58408', '58660', '58643', '58311', '58383', '58492', '58330', '58646', '58541', '58656', '58534', '58354', '58621', '58593', '58597', '58474', '58574', '58647', '58343']
58346: ['58360', '58428', '58553', '58602', '58461', '58414', '58409', '58397', '58351', '58449', '58387', '58458', '58493', '58537', '58422', '58641', '58545', '58525', '58648']
58425: ['58451', '58514']
58356: ['58407', '58659', '58540', '58610', '58603', '58486', '58644', '58440', '58561', '58366', '58570', '58415', '58598', '58596', '58389', '58476', '58467', '58500', '58431']
58363: ['58465', '58579', '58417

,Segment ID,Total Shape Area (km²),Total Depression Area (km²),Depression Percentage (%)
0,58408,1054.530399,32.864939,3.116547
1,58643,147.677300,2.720813,1.842404
2,58308,6503.564399,402.116439,6.183016
3,58346,3897.228101,302.586444,7.764145
4,58425,351.165300,6.384473,1.818082
5,58356,2119.928700,167.096324,7.882167
6,58363,2297.742300,229.341313,9.981159
7,58418,1816.692700,106.027769,5.836307
8,58290,5545.701800,421.691453,7.603933
9,58328,4402.772600,237.857737,5.402453


In [10]:
with open(upstream_seg_list, 'w') as f:
    for sublist in upstream_seg:
        for item in sublist:
            f.write(f'{item}\n')

In [11]:
# Optionally, save to a CSV file
# results_df.to_csv('../../geospacial/misc/river_segment_analysis.csv', index=False)